In [1]:
!pip install stanza

In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
import string
import stanza
import json
import re
import nltk


from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
import warnings
warnings.filterwarnings('ignore')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/afnanaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/afnanaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train_positive = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None)
train_negative = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv', sep='\t', header=None)
train = pd.concat([train_positive, train_negative],ignore_index=True )
train.set_axis(['label', 'tweet'], axis=1, inplace=True)

test_positive = pd.read_csv('test_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None)
test_negative = pd.read_csv('test_Arabic_tweets_negative_20190413.tsv', sep='\t', header=None)
test = pd.concat([test_positive, test_negative],ignore_index=True )
test.set_axis(['label', 'tweet'], axis=1, inplace=True)

In [4]:
data = pd.concat([train_positive, train_negative, test_positive, test_negative])
data.columns = ['label', 'text']
data.sample(5)

,label,text
19371,pos,بمناسبة فوز الهلال .. 💙 سحب على آيفون XR📱 رتوي...
15587,neg,📌 عندهم استعداد بأنهم يطلعونك للبحر ويرجعونك ع...
37,neg,مع غيري كتيير 😱
20644,neg,اشتقت ممكن مسلسل ثاني 💔
11150,neg,الحظ بخرب السالفة بعد ويه 😒


# Preprocessing

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56795 entries, 0 to 5767
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   56795 non-null  object
 1   text    56795 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [6]:
stop = set(stopwords.words('Arabic'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [7]:
stop_words = set(stopwords.words('arabic'))

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)


def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub('\\s+', ' ', text)
    text = re.sub('<.*?>',' ', text)
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    text= remove_stopwords(text)

    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

data['text'] = data['text'].apply(clean_text)

In [8]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35263 entries, 0 to 5766
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   35263 non-null  object
 1   text    35263 non-null  object
dtypes: object(2)
memory usage: 826.5+ KB


In [9]:
data.columns = ['label', 'text']
data.sample(5)

,label,text
17407,pos,مجبر حوثي بيروتي 🤣
1257,neg,الشكر لسفاره السعوديه مهي غريبه عليكم لاهل الخ...
6203,neg,نشهد انه ماكمل جميله العفن
2966,neg,ميران
5105,neg,عمري يدو رسايل الفانز وهوا تعبان مستحيل ماياخد...


# pipeline

In [10]:
Tfidf_Vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)
svc = LinearSVC()
pipeline = make_pipeline(Tfidf_Vectorizer, svc)
pipeline.fit(train.tweet, train.label)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('linearsvc', LinearSVC())])

In [11]:
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipeline, test.tweet, test.label)

              precision    recall  f1-score   support

         neg       0.85      0.82      0.84      5768
         pos       0.83      0.86      0.84      5752

    accuracy                           0.84     11520
   macro avg       0.84      0.84      0.84     11520
weighted avg       0.84      0.84      0.84     11520

accuracy: 0.838
